<a href="https://colab.research.google.com/github/KamonohashiPerry/PyTorch/blob/master/PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchのインストール
+ Colabの編集→ノートブックの設定でGPUモードにすることをお忘れなく。

In [0]:
# pip install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl

In [0]:
# pip install torchvision

In [0]:
# pip install tqdm

インストールできたかの確認

In [0]:
# import torch

In [0]:
# torch.tensor([1, 2, 3]).to("cuda:0")

In [6]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [7]:
# drive mean root directory of  google drive
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"/PyTorch

FashionMNIST  PyTorch_2.ipynb  PyTorch.ipynb  Untitled0.ipynb


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

import warnings
warnings.filterwarnings('ignore')

# Chapter5
## Embedding

In [0]:
# 10000種類のトークンを20次元のベクトルで表現
emb = nn.Embedding(10000, 20, padding_idx=0)

# Embedding層への入力はint64のTensor
inp = torch.tensor([1, 2, 5, 2, 10], dtype=torch.int64)

# 出力はfloat32のTensor
out = emb(inp)

In [10]:
out

tensor([[-0.6818, -2.3338,  0.8712,  0.2704, -1.5498, -0.0728,  1.1133,  0.0600,
          0.4576,  1.2866,  0.9042, -1.0572, -0.7488, -1.1120, -0.9695, -0.2845,
         -0.4058, -0.2447,  0.7323,  0.6415],
        [ 0.5350, -0.9210, -0.8986, -0.4993,  1.6079, -0.0089, -0.4565,  0.4869,
          0.2340,  0.0406, -0.7045, -0.9757,  0.7892, -0.3364,  1.3611,  2.2814,
          0.8039,  1.3851,  0.2326,  0.4055],
        [ 0.0333,  1.3406,  0.0337,  0.0873,  0.5999, -0.7792, -0.4762,  0.8514,
          0.3778, -0.3419, -0.8643,  1.5603,  0.9217,  1.5835, -0.3847,  0.4682,
         -0.1953, -0.3180,  3.1917, -1.3123],
        [ 0.5350, -0.9210, -0.8986, -0.4993,  1.6079, -0.0089, -0.4565,  0.4869,
          0.2340,  0.0406, -0.7045, -0.9757,  0.7892, -0.3364,  1.3611,  2.2814,
          0.8039,  1.3851,  0.2326,  0.4055],
        [-1.1298,  2.8610, -0.5192,  1.2282, -0.9699, -0.0634, -1.9837,  0.2633,
          0.0437,  2.2437, -0.7240, -0.0681,  2.5175, -0.1046,  1.4203, -1.5621,
      

In [11]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-12-03 11:37:47--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.5MB/s    in 7.1s    

2019-12-03 11:37:55 (11.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
!tar xf aclImdb_v1.tar.gz

In [14]:
ls

aclImdb/  aclImdb_v1.tar.gz  adc.json  drive/  sample_data/


In [0]:
# テキストデータを前処理するための関数の用意

import glob
import pathlib
import re

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

# 長い文字列をトークンIDのリストに変換する関数
def text2ids(text, vocab_dict):
  # !?以外の記号の削除
  text = remove_marks_regex.sub("", text)
  # !?と単語の間にスペースを挿入
  text = shift_marks_regex.sub(r" \l ", text)
  tokens = text.split()
  # ボキャブラリー含まれないトークンはID=0を割り当てる
  return [vocab_dict.get(token, 0) for token in tokens]

# IDのリストをint64のTensorに変換
def list2tensor(token_idxes, max_len=100, padding=True):
  # 各文章の分割語のトークンの数を制限
  if len(token_idxes) > max_len:
    token_idxes = token_idxes[:max_len]
  n_tokens = len(token_idxes)
  if padding:
    # 数が足りない場合は末尾を0で埋める
    token_idxes = token_idxes + [0]*(max_len - len(token_idxes))
  return torch.tensor(token_idxes, dtype=torch.int64), n_tokens

In [0]:
# Datasetクラスの作成
class IMDBDataset(Dataset):
  def __init__(self, dir_path, train=True,
               max_len=100, padding=True):
    self.max_len = max_len
    self.padding = padding

    path = pathlib.Path(dir_path)
    vocab_path = path.joinpath("imdb.vocab")

    # ボキャブラリファイルを読み込み、行ごとに分割
    self.vocab_array = vocab_path.open().read().strip().splitlines()

    # 単語をキーとし、値がIDのdictを作る
    self.vocab_dict = dict((w, i+1) for (i, w) in enumerate(self.vocab_array))

    if train:
      target_path = path.joinpath("train")

    else:
      target_path = path.joinpath("test")

    pos_files = sorted(glob.glob(str(target_path.joinpath("pos/*.txt"))))
    neg_files = sorted(glob.glob(str(target_path.joinpath("neg/*.txt"))))

    # posは1、negは0のlabelを付けて
    # (file_path, label)のtupleのリストを作成
    self.labeled_files = list(zip([0]*len(neg_files), neg_files)) + list(zip([1]*len(pos_files), pos_files))

  @property
  def vocab_size(self):
    return len(self.vocab_array)

  def __len__(self):
    return len(self.labeled_files)

  def __getitem__(self, idx):
    label, f = self.labeled_files[idx]

    # ファイルのテキストデータを読み取って小文字に変換
    data = open(f).read().lower()

    # テキストデータをIDのリストに変換
    data = text2ids(data, self.vocab_dict)

    # IDのリストをTensorに変換
    data, n_tokens = list2tensor(data, self.max_len, self.padding)

    return data, label, n_tokens


In [19]:
pwd

'/content'

In [0]:
# 訓練用とテスト用のDataLoaderの作成
train_data = IMDBDataset("/content/aclImdb/")
test_data = IMDBDataset("/content/aclImdb/", train=False)

train_loader = DataLoader(train_data, batch_size=32,
                          shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32,
                          shuffle=False, num_workers=4)

In [0]:
# ネットワークの定義と訓練
# 入力XをEmbeddingでベクトルの時系列に変換して、これをRNNに入れて最後に出力が1次元の線形層につなげる

class SequenceTaggingNet(nn.Module):
  def __init__(self, num_embeddings,
               embedding_dim=50,
               hidden_size=50,
               num_layers=1,
               dropout=0.2):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
    self.linear = nn.Linear(hidden_size, 1)

  def forward(self, x, h0=None, l=None):
    # IDをEmbeddingで多次元のベクトルに変換する
    # xは(batch_size, step_size, embedding_size)
    # -> (batch_size, step_size, hidden_dim)
    x = self.emb(x)

    # 初期状態h0とともにRNNにxを渡す
    # xは(batch_size, step_size, embedding_dim)
    # -> (batch_size, step_size, hidden_dim)
    x, h = self.lstm(x, h0)

    # 最後のステップのみ取り出す
    # xは(batch_size, step_size, hidden_dim)
    # -> (batch_size, 1)
    if l is not None:
      # 入力のもともとの長さがある場合はそれを使用する
      x = x[list(range(len(x))), l-1, :]

    else:
      # なければ単純に最後を使用する
      x = x[:, -1, :] 

    # 取り出した最後のステップを線形層に入れる
    x = self.linear(x)

    # 余分な次元を削除する
    # (batch_size, 1) -> (batch_size, )
    x = x.squeeze()
    return x

In [0]:
# 訓練の作成

def eval_net(net, data_loader, device="cpu"):
  net.eval()
  ys = []
  ypreds = []

  for x, y, l in data_loader:
    x = x.to(device)
    y = y.to(device)
    l = l.to(device)

    with torch.no_grad():
      y_pred = net(x, l=l)
      y_pred = (y_pred > 0).long()
      ys.append(y)
      ypreds.append(y_pred)

    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [33]:
# 評価の作成

from statistics import mean

# num_embeddingsには0を含めてtrain_data.vocab+1を入れる
net = SequenceTaggingNet(train_data.vocab_size + 1, num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
  losses = [] # 損失関数の初期化
  net.train() # ニューラルネットの初期化

  for x, y, l in tqdm.tqdm(train_loader):
    # データをGPUに送る
    x = x.to("cuda:0")
    y = y.to("cuda:0")
    l = l.to("cuda:0")

    y_pred = net(x, l=l) # 予測結果
    loss = loss_f(y_pred, y.float()) # 損失関数の計算
    net.zero_grad() # 前回のbackwardメソッドで計算された勾配の値を削除
    loss.backward() # 微分の計算
    opt.step() # 勾配を更新
    losses.append(loss.item()) # 損失関数のappend
  train_acc = eval_net(net, train_loader, "cuda:0") # 訓練における評価
  val_acc = eval_net(net, test_loader, "cuda:0") # バリデーションにおける評価
  print(epoch, mean(losses), train_acc, val_acc)

  0%|          | 0/782 [00:00<?, ?it/s]

0 0.6893604791835141 0.53125 0.0


  0%|          | 0/782 [00:00<?, ?it/s]

1 0.6694488896204688 0.78125 0.5625


  0%|          | 0/782 [00:00<?, ?it/s]

...

7 0.15167732976610437 0.96875 0.75


  0%|          | 0/782 [00:00<?, ?it/s]

8 0.11278490032024129 0.9375 0.71875


100%|██████████| 782/782 [00:16<00:00, 48.05it/s]


9 0.08585180637552915 1.0 0.78125
